In [2]:
import cv2
import os
import numpy as np
import insightface
from tqdm import tqdm
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image


In [1]:
# import torch
import onnxruntime as rt

In [3]:
# Method-1, use FaceAnalysis
app = FaceAnalysis(allowed_modules=['detection'], providers=['CUDAExecutionProvider']) # enable detection model only
app.prepare(ctx_id=0, det_size=(640, 640))

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/chuongh/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/chuongh/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/chuongh/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/chuongh/.insightface/models/buffalo_l/genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/chuongh/.insightface/models/buffalo_l/w600k_r50.onnx recognition
set det-size: (640, 640)


In [2]:
session_fp32 = rt.InferenceSession("/home/chuongh/.insightface/models/buffalo_l/1k3d68.onnx", providers=['CUDAExecutionProvider'])

2023-08-08 20:48:25.525711574 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:566 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Please reference https://onnxruntime.ai/docs/reference/execution-providers/CUDA-ExecutionProvider.html#requirements to ensure all dependencies are met.


In [3]:
session_fp32._providers

['CPUExecutionProvider']

In [3]:
img = cv2.imread('/home/chuongh/vm2m/data/VideoMatte240K/valid/fgr/vm_0000/00000.jpg')
faces = app.get(img)


In [6]:
len(valid_videos)

425

In [7]:
len(invalid_videos)

50

In [4]:
invalid_videos = []
valid_videos = []
import random
VIDEO_PATH = '/mnt/localssd/VideoMatte240K/valid/fgr'
for video in os.listdir(VIDEO_PATH):
    if video in invalid_videos or video in valid_videos:
        continue
    is_invalid = False
    print(video)
    frames = os.listdir(os.path.join(VIDEO_PATH, video))
    frames = random.sample(frames, 20)
    for frame in tqdm(frames):
        if frame.endswith('.jpg'):
            frame_path = os.path.join(VIDEO_PATH, video, frame)
            faces = app.get(cv2.imread(frame_path))
            if len(faces) > 1:
                invalid_videos.append(video)
                is_invalid = True
                break
    if not is_invalid:
        valid_videos.append(video)
    


vm_0000


100%|██████████| 20/20 [00:00<00:00, 21.19it/s]


vm_0100


100%|██████████| 20/20 [00:00<00:00, 22.03it/s]


vm_0200


100%|██████████| 20/20 [00:01<00:00, 12.58it/s]


vm_0300


100%|██████████| 20/20 [00:01<00:00, 12.46it/s]


In [5]:
with open("../vm2m/dataloader/valid_vm240k_valid.txt", "w") as f:
    for video in valid_videos:
        f.write(video + "\n")